In [1]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt

# Exercise i)
Using the methods of characteristics to solve  
$$u_t(x,t) + cu_x(x,t) = 0 \quad u(x,0) = u_0(x)$$
we reformulate to  
$$\frac{du}{d\tau} = \frac{du}{dt}\frac{dt}{d\tau} + \frac{du}{dx}\frac{dx}{d\tau}$$  
and get:  
$\frac{dt}{d\tau} = 1$ so, together with $t(\tau=0) = 0 \quad$ $t = \tau$  
$\frac{dx}{d\tau} = c$ so, together with $x(\tau=0) = \xi \quad$ $x = c\tau + \xi$  
$\frac{du}{d\tau} = 0$ so, together with $u(\tau=0) = u_0(\xi)\quad$ $u = u_0(\xi)$   
Combining all three equations we obtain
$$u(x,t) = u_0(x-ct)$$


In [2]:

def ftfs(south, southeast, c, delta_t, delta_x):
    """
    Calculates the ftfs scheme
    Input:
        south:     The soutern point aka v_i_j
        southeast: v_i+1_j
    
    Output:
        v_i_{j+1}: The newly calculated point
    """
    
    factor = c * delta_t / delta_x
    return (1 - factor) * south + factor * southeast

def ftcs(south, southeast, southwest, c, delta_t, delta_x):
    """
    Calculates the ftcs scheme
    Input:
        south:     v_i_j
        southeast: v_i+1_j
        southwest: v_i-1,j
    
    Output:
        v_i_{j+1}: The newly calculated point
    """
    
    factor = c * delta_t / (2 * delta_x)
    return south - factor * (southeast - southwest)

def upwind(south, southwest, c, delta_t, delta_x):
    """
    Calculates the upwind scheme
    Input:
        south:     v_i_j
        southwest: v_i-1_j
    
    Output:
        v_i_{j+1}: The newly calculated point
    """
    
    factor = c * delta_t / delta_x
    return (1 - factor) * south + factor * southwest

def lax(southeast, southwest, c, delta_t, delta_x):
    """
    Calculates the lax-friedrich scheme
    Input:
        southeast: v_i+1_j
        southwest: v_i-1_j
    
    Output:
        v_i_{j+1}: The newly calculated point
    """
    
    factor = c * delta_t / (2 * delta_x)
    return 0.5 * (southeast + southwest) - factor * (southeast - southwest)

def u_0(x):
    """
    calculate the value of u_0 at x, also takes np.arrays as arguments
    Input:
        x: point(value) or points(as np.array) where to evaluate the function
    """
    return np.exp(-10 * (x - 2)**2)

# Approximate solutions

In [21]:
#setting the hyperparameters
T = 150 # maximum number of the time, here the max we should plot is 150
omega_boundaries = (0,10)
delta_t = 0.05
delta_x = 0.05
c = 0.5

In [22]:
#calculate the initial solution array which is the same for every scheme

def give_initial_solution(delta_x, delta_t, c):

    num_points_time = int(T/delta_t)+1
    num_points_space = int((omega_boundaries[1] - omega_boundaries[0])/delta_x)+1
    sol_array = np.empty((num_points_time, num_points_space))
    sol_array[:] = np.nan

    sol_at_time_zero = u_0(np.arange(omega_boundaries[0], num_points_space)*delta_x)
    sol_at_left_border = u_0(-1*c*np.arange(0, num_points_time)*delta_t)
    sol_at_right_border = u_0(10 - c*np.arange(0, num_points_time)*delta_t)

    sol_array[0,:] = sol_at_time_zero
    sol_array[:,0] = sol_at_left_border
    sol_array[:,-1] = sol_at_right_border
    
    return sol_array


In [19]:
# calculate the solution arrays

def calculate_ftfs(delta_x, delta_t, c):
    ftfs_sol = give_initial_solution(delta_x, delta_t, c)
    for j in range(1, ftfs_sol.shape[0]):
        for i in range(1, ftfs_sol.shape[1]-1):
            ftfs_sol[j][i] = ftfs(ftfs_sol[j-1][i], ftfs_sol[j-1][i+1], c, delta_t, delta_x)
    return ftfs_sol

def calculate_ftcs(delta_x, delta_t, c):
    ftcs_sol = give_initial_solution(delta_x, delta_t, c)
    for j in range(1, ftcs_sol.shape[0]):
        for i in range(1, ftcs_sol.shape[1]-1):
            ftcs_sol[j][i] = ftcs(ftcs_sol[j-1][i], ftcs_sol[j-1][i+1], ftcs_sol[j-1][i-1], c, delta_t, delta_x)
    return ftcs_sol

def calculate_upwind(delta_x, delta_t, c):
    upwind_sol = give_initial_solution(delta_x, delta_t, c)
    for j in range(1, upwind_sol.shape[0]):
        for i in range(1, upwind_sol.shape[1]-1):
            upwind_sol[j][i] = upwind(upwind_sol[j-1][i], upwind_sol[j-1][i-1], c, delta_t, delta_x)
    return upwind_sol

def calculate_lax(delta_x, delta_t, c):
    lax_sol = give_initial_solution(delta_x, delta_t, c)
    for j in range(1, lax_sol.shape[0]):
        for i in range(1, lax_sol.shape[1]-1):
            lax_sol[j][i] = lax(lax_sol[j-1][i+1], lax_sol[j-1][i-1], c, delta_t, delta_x)
    return lax_sol

In [20]:
fig, axes = plt.subplots(5, 4, figsize = (50,40), dpi=200)

t_array = [0, 10, 50, 100, 150]
xx = np.arange(omega_boundaries[0], omega_boundaries[1]+delta_x, delta_x)
func_array = [calculate_ftfs, calculate_ftcs, calculate_upwind, calculate_lax]
for i in range(5):
    for j in range(4):
        res = func_array[j](delta_x, delta_t, c)
        #print(np.isnan(res[t_array[i]]).any())
        yy_num = res[int(t_array[i]/delta_t)]
        yy_exact = u_0(xx - c*t_array[i])
        axes[i][j].plot(xx, yy_num, label="Numerical approximation")
        axes[i][j].plot(xx, yy_exact, label="Exact solution")
        axes[i,j].legend()
fig.suptitle('Solutions of the numerical simulations at t = 0, 10, 50, 100, 150')
fig.text(0.5, 0.1, 'x', ha='center', va='center')
fig.text(0.09, 0.5, 'Value of the function', ha='center', va='center', rotation='vertical')
fig.text(0.11, 0.8, 't=0', ha='center', va='center', rotation='vertical')
fig.text(0.11, 0.65, 't=10', ha='center', va='center', rotation='vertical')
fig.text(0.11, 0.50, 't=50', ha='center', va='center', rotation='vertical')
fig.text(0.11, 0.35, 't=100', ha='center', va='center', rotation='vertical')
fig.text(0.11, 0.2, 't=150', ha='center', va='center', rotation='vertical')
fig.text(0.2, 0.9, 'FTFS' , ha='center', va='center')
fig.text(0.4, 0.9, 'FTCS' , ha='center', va='center')
fig.text(0.6, 0.9, 'Upwind' , ha='center', va='center')
fig.text(0.8, 0.9, 'Lax' , ha='center', va='center')
plt.show()

<ipython-input-2-d12f722c1975>:28: RuntimeWarning: overflow encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: invalid value encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: overflow encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: invalid value encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: overflow encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: invalid value encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipython-input-2-d12f722c1975>:28: RuntimeWarning: overflow encountered in double_scalars
  return south - factor * (southeast - southwest)
<ipyt

The y axis is not normalized. I kept that on purpose so one can see the whole behaviour of the solution.  
What you can observe (with a fulfilled CFL condition) is that as predicted the FTFS and FTFS schemes behave unstable. The former propagating the hill in the solution function to the left instead of the right direction (or being 0) and the latter developing higher and higher oscillations (for that look at the scale , for $\Delta t,\Delta x = 0.01$ it goes up to 1e225)  
The other two schemes converge to the exact solution, where the upwind scheme performs uniformly better on all observations where the CFL condition is fulfilled. (I looked at $\Delta t,\Delta x = 0.1, 0.05, 0.02, 0.01$)  
When the CFL-condition is not fulfilled, we also observe oscilations in the upwind and Lax scheme, although still less then in the FTCS scheme. (Due to not scaling the y-axis the exact solution seems to be a straight line, although it is not)  
As for the mysterious thing that should happen, I don't know what it should be